## Group 4 Project
### Members: Austin Mallie, Cynthia Portales-Loebell, Graham Ward
### Course: ADS507 - Fundamentals of Data Engineering

In [ ]:
# Import the necessary libraries
import os
import requests
import mysql.connector
import datetime
import pandas as pd
import matplotlib.pyplot as plt

from dotenv import load_dotenv
from IPython.display import HTML, display
from openpyxl import Workbook
from openpyxl.drawing.image import Image
from datetime import datetime, timedelta
from mysql.connector import Error

In [ ]:
# Load environment variables from .env file
load_dotenv()

In [ ]:
# Set API keys as environment variables
TICKETMASTER_API_KEY = os.getenv('TICKETMASTER_API_KEY')
TOMORROW_API_KEY = os.getenv('TOMORROW_API_KEY')

### may not need right here, will check once all code has been moved into this file
TICKETMASTER_Events_API_Endpoint = "https://app.ticketmaster.com/discovery/v2/events.json"
TOMORROW_API_Endpoint = "https://api.tomorrow.io/v4/weather/forecast"

In [3]:
# Set the SQL Connection environment variables
MYSQL_HOST = os.getenv('MYSQL_HOST')
MYSQL_USER = os.getenv('MYSQL_USER')
MYSQL_PASSWORD = os.getenv('MYSQL_PASSWORD')
MYSQL_DATABASE = os.getenv('MYSQL_DATABASE')

### Cynthias Events and Venues pulling section

In [ ]:
conn = mysql.connector.connect(
    host=MYSQL_HOST,
    user=MYSQL_USER,
    password=MYSQL_PASSWORD,
    database=MYSQL_DATABASE
)

In [ ]:
# In order to pull the next 7 days of events
# Get today's date
today = datetime.today()

# Get the date 6 days from today
six_days_later = today + timedelta(days=6)

# Format the dates in the correct format (e.g., 'YYYY-MM-DD')
start_date = today.strftime('%Y-%m-%dT00:00:00Z')
end_date = six_days_later.strftime('%Y-%m-%dT23:59:59Z')

In [ ]:
# EVENTS API
# API pull with parameters
params = {
    "apikey": TICKETMASTER_API_KEY,       
    "city": "San Diego", 
    "size": 200,
    "startDateTime": start_date,  # Start date (today)
    "endDateTime": end_date      # End date (7 days later)                 
}

# Send the request to the API
response = requests.get(TICKETMASTER_Events_API_Endpoint, params=params)

In [ ]:
#Establish connection
cursor = conn.cursor()

if conn.is_connected():
    print("Connection is active")
else:
    print("Connection is not active")

In [ ]:
# Check API response is successful
if response.status_code == 200:
    print("API call successful!")  
    
    # Process the response data
    data = response.json()
    events = data.get("_embedded", {}).get("events", [])

    # Convert data to a DataFrame
    df = pd.DataFrame(events)

In [ ]:
# Review the first 10 rows of the DataFrame before further processing
#limit column width for table
pd.set_option('display.max_colwidth', 40)  

# View the rows
df.head(5)

In [ ]:
# Extract Event details
df['event_id'] = df['id']
df['event_name'] = df['name']
df['event_start_date'] = df['dates'].apply(lambda x: x['start']['localDate'] if isinstance(x, dict) and 'start' in x else None)
df['event_category'] = df['classifications'].apply(lambda x: x[0]['segment']['name'] if isinstance(x, list) and len(x) > 0 and 'segment' in x[0] else None
)

In [ ]:
# Extract Venue details
df['venue_id'] = df['_embedded'].apply(lambda x: x['venues'][0]['id'] if isinstance(x, dict) and 'venues' in x else None)
df['venue_name'] = df['_embedded'].apply(lambda x: x['venues'][0]['name'] if isinstance(x, dict) and 'venues' in x else None)
df['venue_city'] = df['_embedded'].apply(lambda x: x['venues'][0]['city']['name'] if isinstance(x, dict) and 'venues' in x else None)
df['venue_state'] = df['_embedded'].apply(lambda x: x['venues'][0]['state']['stateCode'] if isinstance(x, dict) and 'venues' in x and 'state' in x['venues'][0] else None)
df['venue_zip'] = df['_embedded'].apply(lambda x: x['venues'][0].get('postalCode') if isinstance(x, dict) and 'venues' in x else None)
df['venue_country'] = df['_embedded'].apply(lambda x: x['venues'][0]['country']['countryCode'] if isinstance(x, dict) and 'venues' in x else None)
df['venue_location'] = df['_embedded'].apply(lambda x: f"{x['venues'][0]['location']['latitude']},{x['venues'][0]['location']['longitude']}" if isinstance(x, dict) and 'venues' in x and 'location' in x['venues'][0] else None
)

In [ ]:
# Prepare data for insertion
events_df = df[['event_id', 'event_name', 'event_start_date', 'venue_id', 'event_category']]
venues_df = df[['venue_id', 'venue_name', 'venue_city', 'venue_state', 'venue_country', 'venue_location']].drop_duplicates() #only show each venue once

print(f"Number of rows pulled from API: {len(events_df)}")

In [ ]:
# Insert data to the database or provide failure status
try:
    cursor = conn.cursor()

    # Before loading in new data, delete event data that is in the past now since the last pull
    delete_past_events_query = "DELETE FROM events WHERE start_date < CURDATE();"
    cursor.execute(delete_past_events_query)
    deleted_events_count = cursor.rowcount  # Get the number of deleted rows
    conn.commit()  # Commit deletion
    print(f"Count of past events deleted from MySQL: {deleted_events_count}")  

    # Insert venues (ignore duplicates)
    insert_venue_query = """
    INSERT INTO venues (venue_id, name, city, state, country, location)
    VALUES (%s, %s, %s, %s, %s, %s)
    ON DUPLICATE KEY UPDATE 
        name=VALUES(name), 
        city=VALUES(city), 
        state=VALUES(state),
        country=VALUES(country),
        location=VALUES(location);
    """

    for _, row in venues_df.iterrows():
        cursor.execute(insert_venue_query, tuple(row))
    conn.commit()  # Commit after inserting venues

    # Insert events (ignore duplicates)
    insert_event_query = """
    INSERT INTO events (event_id, name, start_date, venue_id, category)
    VALUES (%s, %s, %s, %s, %s)
    ON DUPLICATE KEY UPDATE 
        name=VALUES(name),
        start_date=VALUES(start_date),
        venue_id=VALUES(venue_id),
        category=VALUES(category);
    """

    for _, row in events_df.iterrows():
        cursor.execute(insert_event_query, tuple(row))
    conn.commit()  # Commit after inserting events

        
    print(f"Inserted {cursor.rowcount} records into MySQL!")

except Error as e:
    print(f"Error: {e}")

else:
    print(f"API call succeeded with status {response.status_code}")

finally:
    if 'conn' in locals() and conn.is_connected():
        cursor.close()
        print("MySQL connection all set!")

In [ ]:
# Group by 'venue_name' and 'event_id' to ensure distinct events are counted
event_counts = df.groupby('venue_name')['event_id'].nunique().reset_index()

# Rename columns for better readability
event_counts.columns = ['Venue', 'Number of Events']

# Sort the DataFrame in descending order by the number of events
event_counts = event_counts.sort_values(by='Number of Events', ascending=False)

# Create a bar plot to visualize the number of events at each venue
plt.figure(figsize=(12, 8))
plt.bar(event_counts['Venue'], event_counts['Number of Events'], color='skyblue')
plt.xticks(rotation=90)
plt.title('Number of Events at Each Venue for the next 7 days')
plt.xlabel('Venues')
plt.ylabel('Number of Events')
plt.xticks(rotation=45, ha='right')
plt.tight_layout()
plt.show()

In [ ]:
# Count the number of missing values in the 'venue_name' column
missing_venue_name_count = df['venue_name'].isna().sum()

# Print the count of missing data
print(f"Number of missing 'venue_name' values: {missing_venue_name_count}")

In [ ]:
# Group the events by classification and count the number of events in each category
classification_event_count = events_df['event_category'].value_counts()

# Create a bar plot
plt.figure(figsize=(6, 4))
classification_event_count.plot(kind='bar', color='purple')

# Set titles and labels
plt.title('Number of Events by Event Classification', fontsize=16)
plt.xlabel('Event Classification', fontsize=12)
plt.ylabel('Number of Events in Next 7 days', fontsize=12)
plt.xticks(rotation=45, ha='right')  

# Display the plot
plt.tight_layout()  
plt.show()

### Austins Weather API calls section

In [ ]:
# Get today's date
today = datetime.today()

# Get the date 7 days from today
next_7_days = today + timedelta(days=6)

# Format the dates in the correct format (e.g., 'YYYY-MM-DD')
start_date = today.strftime('%Y-%m-%dT00:00:00Z')
end_date = next_7_days.strftime('%Y-%m-%dT23:59:59Z')

print("Start Date:", start_date)
print("End Date:", end_date)

In [ ]:
# Weather API
# Load the API key from environment variables
TOMORROW_API_KEY = os.getenv('TOMORROW_API_KEY')

# API pull with parameters
params = {
    "apikey": TOMORROW_API_KEY,       
    "city": "San Diego", 
    "startDateTime": start_date,  # Start date (today)
    "endDateTime": end_date      # End date (7 days later)                
}

# Define the API endpoint
TOMORROW_API_Endpoint = 'https://api.tomorrow.io/v4/weather/forecast?location=San%20Diego&fields=temperature_2m,weather_code&apikey=8zUbskgrkdcP29AtpCqw8XBCutnp5fJh'

# Send the request to the API
response = requests.get(TOMORROW_API_Endpoint, params=params)

In [ ]:
# Check if the request was successful
if response.status_code == 200:
    print("API call succeeded!")
    data = response.json()  # The API response in JSON format
    print(data)             # Print the data (you'll see the events info here)
else:
    print(f"API call failed with status code: {response.status_code}")
    print("Response details:", response.text)  # Print more details on the failure

In [ ]:
# Extract temperature, humidity, rain, and weather code for each hour
weather_data = []

# Weather code to integer mapping (direct integers from API)
weather_code_icons = {
    1100: "mostly_clear_day.svg",  # Clear or mostly clear
    1101: "partly_cloudy.svg",     # Partly cloudy
    1102: "mostly_cloudy.svg",     # Mostly cloudy
    1001: "cloudy.svg",            # Cloudy
    2100: "fog_light.svg",         # Light fog
    2000: "fog.svg",               # Fog
    4000: "drizzle.svg",           # Drizzle
    4200: "rain_light.svg",        # Light rain
    4001: "rain.svg",              # Rain
    4201: "rain_heavy.svg",        # Heavy rain
    6000: "freezing_drizzle.svg",  # Freezing drizzle
    6200: "freezing_rain_light.svg",  # Light freezing rain
    6001: "freezing_rain.svg",     # Freezing rain
    6201: "freezing_rain_heavy.svg",  # Heavy freezing rain
    8000: "tstorm.svg",            # Thunderstorm
}

# Base URL for weather icons
base_url = "https://raw.githubusercontent.com/tomorrow-io-api/tomorrow-weather-codes/main/icons/"

# Parse the data
for interval in data['timelines']['hourly']:
    time = interval['time']
    temperature = interval['values']['temperature']
    temperatureApparent = interval['values']['temperatureApparent']
    humidity = interval['values']['humidity']
    rain = interval['values']['rainIntensity']
    windSpeed = interval['values']['windSpeed']
    weather_codeday = interval['values']['weatherCode']  # Integer weather code

    # Get the icon filename based on the integer weather code
    icon_filename = weather_code_icons.get(weather_codeday, "default.png")

    # Construct the full icon URL
    weather_icon = f"{base_url}{icon_filename}"

    weather_data.append({
        'time': time,
        'temperature': temperature,
        'temperatureApparent': temperatureApparent,
        'humidity': humidity,
        'rain': rain,
        'windSpeed': windSpeed,
        'weather_code': weather_codeday,
        'weather_icon': weather_icon
    })

# Convert values to a DataFrame for easier visualization
weather_df = pd.DataFrame(weather_data)
print(weather_df)

# Function to display icons in the DataFrame
def path_to_image_html(path):
    return f'<img src="{path}" width="50" height="50">'

# Apply the function to the 'weather_icon' column
weather_df['weather_icon'] = weather_df['weather_icon'].apply(path_to_image_html)

# Display the DataFrame with icons
display(HTML(weather_df.to_html(escape=False)))

In [ ]:
# Function to connect to MySQL database
def connect_to_mysql():
    try:
        connection = mysql.connector.connect(
            host=MYSQL_HOST,  # Use variables loaded from dotenv
            database=MYSQL_DATABASE,
            user=MYSQL_USER,
            password=MYSQL_PASSWORD,
        )
        if connection.is_connected():
            print("Connected to MySQL database")
            return connection
    except Error as e:
        print(f"Error: {e}")
        return None
    
# Convert 'time' from 'YYYY-MM-DDTHH:MM:SSZ' to 'YYYY-MM-DD HH:MM:SS'
weather_df['time'] = pd.to_datetime(weather_df['time']).dt.strftime('%Y-%m-%d %H:%M:%S')

In [ ]:
# Function to insert data into MySQL
def insert_data_to_mysql(df, connection):
    try:
        if connection.is_connected():
            cursor = connection.cursor()
            insert_query = """
                INSERT INTO weather_forecasts (
                    forecast_time, forecast_temperature, forecast_temp_apparent, forecast_humidity, rain_intensity, forecast_winds, weather_code, weather_icon
                )
                VALUES (%s, %s, %s, %s, %s, %s, %s, %s)
            """
            # Insert each row from the DataFrame
            for row in df.itertuples(index=False):
                cursor.execute(insert_query, (
                    row.time, 
                    row.temperature, 
                    row.temperatureApparent, 
                    row.humidity, 
                    row.rain, 
                    row.windSpeed, 
                    row.weather_code, 
                    row.weather_icon
                ))
            connection.commit()
            print(f"Inserted {cursor.rowcount} rows successfully")
    except Error as e:
        print(f"Error during data insertion: {e}")
    finally:
        if connection.is_connected():
            cursor.close()
            connection.close()
            print("MySQL connection is closed")

# Ensure connection is established
mysql_connection = connect_to_mysql()




if mysql_connection:
    insert_data_to_mysql(weather_df, mysql_connection)
else:
    print("Failed to connect to MySQL database.")